# Анализ NPS среди клиентов телекоммуникационной компании

**Цель исследования** — определить текущий уровень потребительской лояльности NPS среди клиентов из России. 

**Заказчик исследования** — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. 

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
# По данным запроса создадим датафрейм, который содержит основную информацию о клиентах
query = """
   select 
    user_id,
    lt_day,
    CASE
        WHEN lt_day <= 365 THEN TRUE
        WHEN lt_day > 365 THEN FALSE
        ELSE NULL
    END as is_new,
    age,
    CASE
        WHEN gender_segment = 1.0 THEN "женский"
        WHEN gender_segment = 0.0 THEN "мужской"
        ELSE NULL
    END AS gender_segment,
    os_name,
    cpe_type_name,
    loc.country,
    loc.city,
    substr(ag.title, 4) as age_segment,
    substr(tr.title, 4) as traffic_segment,
    substr(lt.title, 4) as lifetime_segment,
    nps_score,
    CASE
        WHEN nps_score <= 6 THEN "критики"
        WHEN nps_score >= 7 and nps_score <= 8 THEN "нейтралы"
        WHEN nps_score >= 9 and nps_score <= 10 THEN "cторонники"
        ELSE NULL
    END as nps_group
from user as u
LEFT JOIN location AS loc ON u.location_id = loc.location_id
LEFT JOIN age_segment AS ag ON u.age_gr_id = ag.age_gr_id
LEFT JOIN traffic_segment AS tr ON u.tr_gr_id = tr.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id;
"""
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,женский,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,0,53.0,мужской,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,0,57.0,мужской,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники
3,A004TB,4190,0,44.0,женский,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,cторонники
4,A004XT,1163,0,24.0,мужской,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,cторонники
5,A005O0,5501,0,42.0,женский,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,0,45.0,мужской,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,cторонники
7,A009KS,313,1,35.0,мужской,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,cторонники
8,A00AES,3238,0,36.0,женский,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,cторонники
9,A00F70,4479,0,54.0,женский,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,cторонники


In [4]:
# Выгрузим полученные данные в CSV-формате для дальнейшей их обработки в Tableau
df.to_csv('telecomm_csi_tableau_chernyakid.csv', index=False)